# Read CSV

In [1]:
import pandas as pd
import numpy as np

# Italian league

In [2]:
#data = pd.read_csv("SerieA-B/Serie_A-B_stats.csv")

# Dutch league

In [3]:
import os
data_folder = "Eredivisie"
data_frames = []
for data_file in os.listdir(data_folder):
    if data_file[-4:] == ".csv":
        data_frames.append(pd.read_csv(os.path.join(data_folder,data_file)))
        
data = pd.concat(data_frames)

In [4]:
data.head(1)

,AwayTeam,B365A,B365D,B365H,BSA,BSD,BSH,BWA,BWD,BWH,...,SJH,Unnamed: 58,Unnamed: 59,Unnamed: 60,VCA,VCD,VCH,WHA,WHD,WHH
0,Feyenoord,2.1,3.4,3.4,1.91,3.5,3.75,1.91,3.5,3.4,...,3.4,NaN,NaN,NaN,1.95,3.5,3.75,1.83,3.5,3.75


# Set Index

In [5]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%y')
all_data = data.set_index('Date')

# Select features

In [10]:
data = all_data[['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']] 
data.head(5)

,HomeTeam,AwayTeam,FTR,FTHG,FTAG
Date,,,,,
2011-08-05,Excelsior,Feyenoord,A,0.0,2.0
2011-08-06,Heerenveen,Nijmegen,D,2.0,2.0
2011-08-06,Roda,Groningen,H,2.0,1.0
2011-08-06,VVV Venlo,Utrecht,D,0.0,0.0
2011-08-06,Waalwijk,Heracles,D,2.0,2.0


In [11]:
data.tail(5)

,HomeTeam,AwayTeam,FTR,FTHG,FTAG
Date,,,,,
2016-10-01,Vitesse,Groningen,H,2.0,1.0
2016-10-02,Ajax,Utrecht,H,3.0,2.0
2016-10-02,Heracles,Twente,D,1.0,1.0
2016-10-02,Sparta Rotterdam,AZ Alkmaar,D,1.0,1.0
2016-10-02,Willem II,Feyenoord,A,0.0,2.0


In [12]:
input_features = data

In [13]:
g = input_features.columns.to_series().groupby(data.dtypes).groups

In [14]:
object_features = {key.name: value for key, value in g.items()}['object']

In [15]:
object_features

['HomeTeam', 'AwayTeam', 'FTR']

# Create performance history per team

1. Track winning or losing streak as feature
2. Track total wins, losses and draws

In [16]:
class Stats:
    def __init__(self):
        self.win_streak = 0
        self.lose_streak = 0
        self.total_wins = 0
        self.total_losses = 0
        self.total_draws = 0
        self.total_games = 0
        self.previous_encounters = {}
        self.result_series = []
        
    def get_win_ratio(self):
        try:
            return (float(self.total_wins) / float(self.total_games))
        except:
            return(0)
        
    def get_win_ratio_against_opponent(self, opponent):
        try:
            results = self.previous_encounters[opponent]
            return sum(results)
        except:
            return(0)
        
    def average_goals_scored_and_conceded(self, number_of_games):
        if self.total_games == 0:
            return (0,0)
        elif number_of_games > self.total_games:
            number_of_games = self.total_games
        games = self.result_series[-number_of_games:]
        goals_scored = 0
        goals_conceded = 0
        for game in games:
            goals_scored = goals_scored + game["goals_scored"]
            goals_conceded = goals_conceded + game["goals_conceded"]
            
        return (goals_scored/number_of_games, goals_conceded/number_of_games)
    
    def days_since_last_match(self,today):
        if len(self.result_series) == 0:
            return 365
        last_match_date = self.result_series[-1]["date"]
        return((today-last_match_date).days)
            
    #Results: 1 = win, 0 = draw, -1 = loss for this team against the opponent
    def update_previous_encounters(self, opponent, result):
        if opponent in self.previous_encounters:
            self.previous_encounters[opponent].append(result)
        else:
            self.previous_encounters[opponent] = [result]

    def update_result(self, opponent, goals_scored, goals_conceded, date):
        if pd.isnull(goals_scored) or pd.isnull(goals_conceded):
            return False
        result = 2
        if(goals_scored > goals_conceded):
            result = 1
            self.update_win()
        elif(goals_scored < goals_conceded):
            result = -1
            self.update_loss()
        else:
            result = 0
            self.update_draw()
        self.result_series.append({"result":result, "goals_scored":goals_scored, "goals_conceded":goals_conceded, "date":date})
        self.update_previous_encounters(opponent, result)
    
    def update_win(self):
        self.win_streak = self.win_streak + 1
        self.lose_streak = 0
        self.total_wins = self.total_wins + 1
        self.total_games = self.total_games + 1
        
    def update_loss(self):
        self.lose_streak = self.lose_streak + 1
        self.win_streak = 0
        self.total_losses = self.total_losses + 1
        self.total_games = self.total_games + 1
    
    def update_draw(self):
        self.total_draws = self.total_draws + 1
        self.win_streak = 0
        self.lose_streak = 0
        self.total_games = self.total_games + 1

In [17]:
teams = input_features.HomeTeam.unique()
stats = {}
for team in teams:
    stats[team] = Stats()

## Create feature arrays as columns for data frame

In [18]:
feature_names = ["win_streak",
    "lose_streak",
    "total_wins",
    "total_losses",
    "total_draws",
    "win_ratio",
    "opponent_win_ratio",
    "total_games",
    "goals_scored",
    "goals_conceded",
    "days_since_last_match"]

home_results = {}
away_results = {}

for feature in feature_names:
    home_results[feature] = []
    away_results[feature] = []

## Add aggregated stats from previous matches as features then update stats based on this match

In [19]:
def update_results(results, stats, opponent,date):
    results["win_streak"].append(stats.win_streak)
    results["lose_streak"].append(stats.lose_streak)
    results["total_wins"].append(stats.total_wins)
    results["total_losses"].append(stats.total_losses)
    results["total_draws"].append(stats.total_draws)
    results["win_ratio"].append(stats.get_win_ratio())
    results["opponent_win_ratio"].append(stats.get_win_ratio_against_opponent(opponent))
    results["total_games"].append(stats.total_games)
    goals_scored, goals_conceded = stats.average_goals_scored_and_conceded(3)
    if pd.isnull(goals_scored):
        print(stats.result_series[-3:])
    results["goals_scored"].append(goals_scored)
    results["goals_conceded"].append(goals_conceded)
    results["days_since_last_match"].append(stats.days_since_last_match(date))
    
    return results
    

for date, match in input_features.iterrows():
    home_team = match.HomeTeam
    home_stats = stats[home_team]
    
    away_team = match.AwayTeam
    away_stats = stats[away_team]
    
    home_results = update_results(home_results, home_stats, away_team, date)

    away_results = update_results(away_results, away_stats, home_team, date)
    
    #Update stats based on results of this match
    match_result = match.FTR    
    home_goals = match.FTHG
    away_goals = match.FTAG
        
    """if home_team == "Ajax":
        #print("total wins: " + str(home_stats.total_wins))
        #print("total games: " + str(home_stats.total_games))
        #print("win ratio: " + str(home_stats.get_win_ratio_against_opponent(away_team)))
        print("today: " + str(date))
        print("days since last match " + str(home_stats.days_since_last_match(date)))
    elif away_team == "Ajax":
        #print("total wins: " + str(away_stats.total_wins))
        #print("total games: " + str(away_stats.total_games))
        #print("win ratio: " + str(away_stats.get_win_ratio_against_opponent(home_team)))
        print("today: " + str(date))
        print("days since last match " + str(away_stats.days_since_last_match(date)))"""
        
    home_stats.update_result(away_team,home_goals,away_goals, date)
    away_stats.update_result(home_team,away_goals,home_goals, date)

In [20]:
    """if home_team == "Juventus" and away_team == "Roma":
        print("CHECK")
    if home_team == "Roma" and away_team == "Juventus":
        print("CHECK")"""

'if home_team == "Juventus" and away_team == "Roma":\n    print("CHECK")\nif home_team == "Roma" and away_team == "Juventus":\n    print("CHECK")'

In [21]:
columns = {}

for column_name, column  in home_results.iteritems():
    columns["home_" + column_name] = column
 
for column_name, column  in away_results.iteritems():
    columns["away_" + column_name] = column

In [22]:
input_features = pd.DataFrame(data=columns, index=data.index.values)

In [23]:
#input_features = input_features.drop(["away_total_losses", "home_total_losses", "away_total_wins", "home_total_wins", "away_total_draws", "home_total_draws"], 1)
input_features = input_features[['home_win_streak', 'away_win_streak', 'home_lose_streak', 'away_lose_streak', 'home_win_ratio', 'away_win_ratio', 'home_days_since_last_match', 'away_days_since_last_match', 'home_opponent_win_ratio', 'away_opponent_win_ratio']]

In [24]:
input_features.head(10)

,home_win_streak,away_win_streak,home_lose_streak,away_lose_streak,home_win_ratio,away_win_ratio,home_days_since_last_match,away_days_since_last_match,home_opponent_win_ratio,away_opponent_win_ratio
2011-08-05,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-06,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-06,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-06,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-06,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-07,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-07,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-07,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-07,0,0,0,0,0.0,0.0,365,365,0,0
2011-08-13,1,1,0,0,1.0,1.0,8,7,0,0


# Remove first half of a season so all stats have something to work with

In [25]:
#Season has 18 teams
#Each team plays 36 matches
#after 36 games each team should have played two games
buffer_games = 36
input_features = input_features[buffer_games:]

In [26]:
input_features.head(5)

,home_win_streak,away_win_streak,home_lose_streak,away_lose_streak,home_win_ratio,away_win_ratio,home_days_since_last_match,away_days_since_last_match,home_opponent_win_ratio,away_opponent_win_ratio
2011-09-10,2,0,0,1,0.75,0.50,13,15,0,0
2011-09-10,1,2,0,0,0.25,0.50,13,14,0,0
2011-09-10,0,1,1,0,0.00,0.50,13,14,0,0
2011-09-10,0,1,1,0,0.25,0.75,14,15,0,0
2011-09-10,0,4,3,0,0.25,1.00,14,14,0,0


# Check for unusable fields

In [27]:
column_names = list(input_features.columns.values)

## Infinite values

In [28]:
for column in column_names:
    total_infinite_values = sum(np.isinf(input_features[column]))
    print (column + ": " + str(total_infinite_values))

home_win_streak: 0
away_win_streak: 0
home_lose_streak: 0
away_lose_streak: 0
home_win_ratio: 0
away_win_ratio: 0
home_days_since_last_match: 0
away_days_since_last_match: 0
home_opponent_win_ratio: 0
away_opponent_win_ratio: 0


In [29]:
for column in column_names:
    total_NaN_values = sum(np.isnan(input_features[column]))
    print (column + ": " + str(total_NaN_values))

home_win_streak: 0
away_win_streak: 0
home_lose_streak: 0
away_lose_streak: 0
home_win_ratio: 0
away_win_ratio: 0
home_days_since_last_match: 0
away_days_since_last_match: 0
home_opponent_win_ratio: 0
away_opponent_win_ratio: 0


#### Full time result as output
### First convert output to numeric for easyer processing

In [30]:
def ResultToNumeric(results):
    numericResults = []
    for result in results:
        if result == 'H':
            numericResults.append(0)
        elif result == 'A':
            numericResults.append(1)
        else:
            numericResults.append(2)
    return np.array(numericResults)

output = ResultToNumeric(data['FTR'])
output = output[buffer_games:]

# Baseline model

In [31]:
bins = np.bincount(output)
index = np.nonzero(bins)[0]
np.vstack((index,bins[index])).T

array([[  0, 721],
       [  1, 464],
       [  2, 384]], dtype=int64)

## Predict all games as a home win will result in 721/1569=0.460

# Try random forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=1000,
                                                   max_features=4,
                                                   max_depth=None,
                                                   min_samples_split=1,
                                                   n_jobs=-1)

## Scoring

In [33]:
def report_scoring(scores):
    print("List of scores: " + str(scores))
    print("mean scoring: " + str(scores.mean()))
    print("Standard deviation between scorings: " + str(scores.std()))

In [34]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(random_forest_model, input_features, output)

report_scoring(scores)

List of scores: [ 0.47137405  0.47418738  0.47126437]
mean scoring: 0.472275264705
Standard deviation between scorings: 0.00135281125002


## Feature importance

In [35]:
trained_random_forest_model = random_forest_model.fit(input_features,output)
pd.Series(trained_random_forest_model.feature_importances_, input_features.columns)

home_win_streak               0.044315
away_win_streak               0.058243
home_lose_streak              0.059181
away_lose_streak              0.047815
home_win_ratio                0.244800
away_win_ratio                0.236170
home_days_since_last_match    0.094987
away_days_since_last_match    0.095011
home_opponent_win_ratio       0.059568
away_opponent_win_ratio       0.059911
dtype: float64

# Boosted random forest

In [36]:
from sklearn.ensemble import AdaBoostClassifier

boosted_random_forest_model = AdaBoostClassifier(n_estimators=1000)

## Scoring

In [37]:
scores = cross_val_score(boosted_random_forest_model, input_features, output)

report_scoring(scores)

List of scores: [ 0.44274809  0.44933078  0.38888889]
mean scoring: 0.42698925481
Standard deviation between scorings: 0.0270747283409


## Feature importance

In [38]:
trained_boosted_random_forest_model = boosted_random_forest_model.fit(input_features,output)
pd.Series(trained_boosted_random_forest_model.feature_importances_, input_features.columns)

home_win_streak               0.026
away_win_streak               0.018
home_lose_streak              0.010
away_lose_streak              0.020
home_win_ratio                0.391
away_win_ratio                0.427
home_days_since_last_match    0.045
away_days_since_last_match    0.042
home_opponent_win_ratio       0.009
away_opponent_win_ratio       0.012
dtype: float64

# SVM

In [39]:
from sklearn import svm

svm_model = svm.SVC()

## Scoring

In [40]:
scores = cross_val_score(svm_model, input_features, output)

report_scoring(scores)

List of scores: [ 0.46374046  0.46462715  0.48659004]
mean scoring: 0.471652549127
Standard deviation between scorings: 0.0105686010915


# TPOT

In [41]:
from tpot import TPOTClassifier

X_test = input_features.tail(612)
y_test = output[-612]

X_train = input_features[:612]
y_train = output[:612]

tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2)
tpot.fit(X_train, y_train)
#print(tpot.score(X_test, y_test))
tpot.export('tpot_soccer_pipeline.py')

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Generation 1 - Current best internal CV score: 0.594375620257


Generation 2 - Current best internal CV score: 0.595329983709


Generation 3 - Current best internal CV score: 0.597239253375


Generation 4 - Current best internal CV score: 0.597239253375


...

Generation 8 - Current best internal CV score: 0.600365461161


GP Progress:  73%|███████████████▎     | 320/440 [05:33<02:54,  1.45s/pipeline]C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


Generation 9 - Current best internal CV score: 0.600365461161


Generation 10 - Current best internal CV score: 0.600365461161



Best pipeline: GaussianNB(CombineDFs(CombineDFs(input_matrix, input_matrix), Normalizer(input_matrix, 12)))


# TPOT results in Gaussian Naive Bayes

In [46]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, Normalizer

exported_pipeline = make_pipeline(
    make_union(
        make_union(
            FunctionTransformer(lambda X: X),
            FunctionTransformer(lambda X: X)
        ),
        Normalizer(norm="l1")
    ),
    GaussianNB()
)

exported_pipeline.fit(X_train, y_train)

scores = cross_val_score(exported_pipeline, input_features, output)

report_scoring(scores)

List of scores: [ 0.50381679  0.46845124  0.52873563]
mean scoring: 0.500334556302
Standard deviation between scorings: 0.0247338686006
